# 1. Install and import Dependencies

In [1]:
#!pip install torch==1.10.1+cu113 torchvision==0.11.2+cu113 torchaudio===0.10.1+cu113 -f https://download.pytorch.org/whl/cu113/torch_stable.html

Looking in links: https://download.pytorch.org/whl/cu113/torch_stable.html


In [2]:
#!pip install transformers requests beautifulsoup4 pandas numpy

In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re
import numpy as np
import pandas as pd

# 2. Initialize Model

In [2]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

#creates model, link is directly from hugging face transformers
model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

# 3. Encode and Calculate Sentiment

In [3]:
tokens = tokenizer.encode('fantastic, this was great, would recoommend', return_tensors = 'pt')
#must pass tokenized values to the model

In [4]:
tokens
#tokens list is in a list

tensor([[  101, 47088,   117, 10372, 10140, 11838,   117, 11008, 44909, 39639,
         11719, 10163,   102]])

In [5]:
tokenizer.decode(tokens[0])
#so we take the first element to gain the tokens list, and then we decode this to simply show how encode and decode work

'[CLS] fantastic, this was great, would recoommend [SEP]'

In [9]:
result = model(tokens)
result.logits
#torch.argmax(result.logits)
int(torch.argmax(result.logits))+1
#we add 1 as it starts at 0

5

In [10]:
tokens = tokenizer.encode('fantastic, this was great, would recoommend', return_tensors = 'pt')

# 4. Collect Reviews

In [27]:
r = requests.get('https://www.yelp.ca/biz/adda-bistro-toronto?hrid=AuV4ADBfEPtFK0yELEv4pw')
soup = BeautifulSoup(r.text, 'html.parser')

In [28]:
regex = re.compile('.*comment.*')
results = soup.find_all('p', {'class':regex})

In [29]:
reviews = [result.text for result in results]

In [30]:
reviews

["After COVID-19 I was looking for a good restaurant. One of my friend suggested me Adda Bistro's, their food is hygenic and delicio-ous. Highly recommended. Keep the spirits high.",
 'A hidden gem - The food here was delicious and the staff was warm and friendly. Would definitely eat here again. Especially recommend the Egg Devil and the Thandai cheesecake.',
 "I was really happy with Adda Bistro's online ordering process as they have their own vs. having to go through apps like Skip the Dishes. Contactless pickup is great as well. My order was waiting in a labelled brown bag. I would suggest attaching the bill to the outside of the bag to ensure everything is included (it was, but would be helpful to be able to verify upon pickup). My friend and I ordered Chicken Dak, Dhaal Makhni, Steamed Rice, Garlic Naan and Gulab Jamun Cheesecake to share. Everything was delicious and not too spicy. A couple of issues to note: 1. Chicken Dak included 4 small pieces of chicken, 3/4 were unrecogniz

# 5. Load Reviews into DataFrame and Score

In [47]:
list = ["reviews", "hello", "goodluck"]

In [48]:
reviews

["After COVID-19 I was looking for a good restaurant. One of my friend suggested me Adda Bistro's, their food is hygenic and delicio-ous. Highly recommended. Keep the spirits high.",
 'A hidden gem - The food here was delicious and the staff was warm and friendly. Would definitely eat here again. Especially recommend the Egg Devil and the Thandai cheesecake.',
 "I was really happy with Adda Bistro's online ordering process as they have their own vs. having to go through apps like Skip the Dishes. Contactless pickup is great as well. My order was waiting in a labelled brown bag. I would suggest attaching the bill to the outside of the bag to ensure everything is included (it was, but would be helpful to be able to verify upon pickup). My friend and I ordered Chicken Dak, Dhaal Makhni, Steamed Rice, Garlic Naan and Gulab Jamun Cheesecake to share. Everything was delicious and not too spicy. A couple of issues to note: 1. Chicken Dak included 4 small pieces of chicken, 3/4 were unrecogniz

In [49]:
list

['reviews', 'hello', 'goodluck']

In [ ]:
#df = pd.DataFrame(np.array(list), columns = ['reviews'])

In [52]:
df = pd.DataFrame(np.array(reviews), columns = ['reviews'])

In [53]:
df.tail()

,reviews
3,Really innovative idea and approach. The chees...
4,A friend of mine from Scotland was telling me ...
5,"Just Oder food via ""skip the dish"" and it arr..."
6,We have ordered food from them a number of tim...
7,Came into this place because I was craving Ind...


In [62]:
def sentiment(string):
    tokens = tokenizer.encode(string, return_tensors = 'pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

df['sentiment'] = df['reviews'].apply(lambda x: sentiment(x[:512]))

In [63]:
df

,reviews,sentiment
0,After COVID-19 I was looking for a good restau...,5
1,A hidden gem - The food here was delicious and...,5
2,I was really happy with Adda Bistro's online o...,4
3,Really innovative idea and approach. The chees...,5
4,A friend of mine from Scotland was telling me ...,5
5,"Just Oder food via ""skip the dish"" and it arr...",3
6,We have ordered food from them a number of tim...,5
7,Came into this place because I was craving Ind...,5
